#### This basic tutorial shows you how to interface the HC-SR04 Ultrasonic Ranging Module with STM32 Nucleo-L432KC. The datasheet of the HC-SR04 can be found [here](https://cdn.sparkfun.com/datasheets/Sensors/Proximity/HCSR04.pdf). This tutorial is developed based on the tutorial [here](https://www.micropeta.com/video42).

## 1. Wiring

<img src="Figures\STM_pinout.png" alt="STM32 Nucleo-L432KC Pinout" width="800"/>

<img src="Figures\HC-SR04.jpg" alt="HC-SR04" width="500"/>

Connection:
- HC-SR04 Vcc <--> STM32 5V
- HC-SR04 Gnd <--> STM32 GND
- HC-SR04 Trig <--> STM32 PA_4 (A3) (Any pin for GPIO can be used)
- HC-SR04 Echo <--> STM32 PA_5 (A4) (Any pin for GPIO can be used)

## 2. CubeIDE Clock Configuration

We are going to use GPIO and the Timer `TIM1` (see [STM32 datasheet](https://www.st.com/resource/en/datasheet/stm32l432kb.pdf) page 38/156 Table 9) to supply a short 10uS pulse to the Trig pin on the module and record the high level time of the Echo pulse in order to calculate the distance. To set the Timer frequency, we first need to check the HCLK frequency to determine the correct Prescaler to use for the Timer. Open the `.ioc` file in your project and click on the `Clock Configuration` tab to see the clock setting of the STM32. 

<img src="Figures\Clock_setting.png" alt="Clock setting" width="1500"/>

The `APB2 timer clocks (MHz)` is the clock used for `TIM1` (see [STM32 datasheet](https://www.st.com/resource/en/datasheet/stm32l432kb.pdf) page 32/156 Figure 4)

<img src="Figures\Clock_tree.png" alt="Clock tree" width="500"/>

In my case, the `HCLK` is set to $32$ MHz, the `APB2 Prescaler` is set to $1$ and the `Multiplier` is set to $1$, so the `APB2 timer clocks (MHz)` is $32/1 \times 1=32$ MHz. Assume that we want the Timer to run at $1$ MHz, we need to set the Prescaler of the TIM1 to $32$. (i.e., $32/1=1$ MHz) If you want to set the HCLK to a different value, calculate the appropriate Prescaler value for the Timer correspondingly.

## 3. CubeIDE Pinout & Configuration

Go to the `Pinout & Configuration` tab, under `Timers`, select `TIM1`. Set the `Clock Source` to `Internal Clock`. In the Configuration, set the `Prescaler` to $32-1$. (The `Prescaler` value will be $n+1$ where $n$ is the number you enter.) Set pin `PA4` to `GPIO_Output` and label it as `TRIG`, set pin `PA5` to `GPIO_Input` and label it as `ECHO`. (You can Enter User Label by right click on the pin.)

<img src="Figures\ultrasound_pin_config.png" alt="Pin configuration" width="1500"/>

In order to debug, in the `System Core` dropdown list, click `SYS` and make sure the `Debug` mode is set to `Serial Wire`.

<img src="Figures\ultrasound_debug.png" alt="Debug configuration" width="1500"/>

After setting, save the file and generate code.

## 4. Coding

We measure the high level time of the Echo pulse. And compute the distance according to $\text{distance}=\text{high level time} \times 340\text{(m/s)}/2$.

Open the main.c file. Copy the following code and paste between `/* USER CODE BEGIN PV */` and `/* USER CODE END PV */`:

```c
/* USER CODE BEGIN PV */
uint32_t pMillis;
uint32_t Value1 = 0;
uint32_t Value2 = 0;
uint16_t Distance  = 0;  // cm
/* USER CODE END PV */
```

Copy the following code and paste between `/* USER CODE BEGIN 2 */` and `/* USER CODE END 2 */`:

```c
  /* USER CODE BEGIN 2 */
  HAL_TIM_Base_Start(&htim1);
  HAL_GPIO_WritePin(TRIG_GPIO_Port, TRIG_Pin, GPIO_PIN_RESET);  // pull the TRIG pin LOW
  /* USER CODE END 2 */
```

Copy the following code and paste between `/* USER CODE BEGIN WHILE */` and `/* USER CODE END WHILE */`:

```c
  /* USER CODE BEGIN WHILE */
  while (1)
  {
	HAL_GPIO_WritePin(TRIG_GPIO_Port, TRIG_Pin, GPIO_PIN_SET);  // pull the TRIG pin HIGH
	__HAL_TIM_SET_COUNTER(&htim1, 0);
	while (__HAL_TIM_GET_COUNTER (&htim1) < 10);  // wait for 10 us
	HAL_GPIO_WritePin(TRIG_GPIO_Port, TRIG_Pin, GPIO_PIN_RESET);  // pull the TRIG pin LOW

	pMillis = HAL_GetTick();
	// wait 50ms for the echo pin to go high. If timeout, exit the while loop.
	while (!(HAL_GPIO_ReadPin (ECHO_GPIO_Port, ECHO_Pin)) && pMillis + 50 >  HAL_GetTick());
	Value1 = __HAL_TIM_GET_COUNTER (&htim1);

	pMillis = HAL_GetTick();
	// wait 50ms for the echo pin to go low. If timeout, exit the while loop.
	while ((HAL_GPIO_ReadPin (ECHO_GPIO_Port, ECHO_Pin)) && pMillis + 50 > HAL_GetTick());
	Value2 = __HAL_TIM_GET_COUNTER (&htim1);

	Distance = (Value2-Value1)* 0.0343/2;
	HAL_Delay(50);
    /* USER CODE END WHILE */
```

## 5. Debugging

Now, click on `Debug` and make sure the `Enable live expressions` option in the Debugger configuration is selected, which lets us see the changes of the distance.

<img src="Figures\Debugger_configuration_ultrasound.png" alt="Debugger configuration" width="700"/>

After entering the debugging mode, add `Distance` under the `Live Expressions` tab.

<img src="Figures\Dist_change.png" alt="Live expression" width="1000"/>

Now click `Resume` to run the code, and see the values change when you place an object in front of the module.

<img src="Figures\Live_exp_dist.png" alt="Distance change" width="1000"/>